In [521]:
import numpy as np
import pandas as pd
import random

In [522]:
df = pd.read_csv('nutrients_csvfile.csv', encoding='ISO-8859-1')

In [523]:
df.iloc[:, 2:9] = df.iloc[:, 2:9].apply(pd.to_numeric, errors='coerce').fillna(0)
df.head()

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
0,Cows' milk,1 qt.,976.0,660.0,32.0,40.0,36.0,0.0,48.0,Dairy products
1,Milk skim,1 qt.,984.0,360.0,36.0,0.0,0.0,0.0,52.0,Dairy products
2,Buttermilk,1 cup,246.0,127.0,9.0,5.0,4.0,0.0,13.0,Dairy products
3,"Evaporated, undiluted",1 cup,252.0,345.0,16.0,20.0,18.0,0.0,24.0,Dairy products
4,Fortified milk,6 cups,0.0,0.0,89.0,42.0,23.0,1.4,119.0,Dairy products


In [556]:
for row in df:
    data_list = df.values.tolist()
    
category_column = list({item[-1] for item in data_list})

print(category_column)

['Desserts, sweets', 'Fruits R-Z', 'Vegetables R-Z', 'Fruits G-P', 'Seeds and Nuts', 'Drinks,Alcohol, Beverages', 'Dairy products', 'Vegetables F-P', 'Jams, Jellies', 'Fish, Seafood', 'Meat, Poultry', 'Vegetables A-E', 'Breads, cereals, fastfood,grains', 'Fats, Oils, Shortenings', 'Fruits A-F', 'Soups']


In [525]:
def random_num(start, end):
    range_val = (end - start) + 1
    random_int = start + random.randint(0, range_val - 1)
    return random_int

In [557]:
num_group_morning = 3
num_group_lunch = 4
num_group_dinner = 4
num_group_snack = 1
total_group = 12 #num_genes

G1 = ['Dairy products']
G2 = ['Breads, cereals, fastfood,grains']
G3 = ['Fish, Seafood','Meat, Poultry']
G4 = ['Fats, Oils, Shortenings']
G5 = ['Vegetables A-E', 'Vegetables R-Z', 'Vegetables F-P']
G6 = ['Fruits A-F', 'Fruits G-P', 'Fruits R-Z']
G7 = ['Seeds and Nuts']
G8 = ['Soups']
G9 = ['Desserts, sweets', 'Drinks,Alcohol, Beverages', 'Jams, Jellies']

population_size = 25
num_generations = 50
mutation_rate = 0.1

weight = 50
tdee = 2000
aim = 'Giữ cân'
favourite = ['Apple juice canned', 'Beans', 'Salmon', 'Beef', 'Corn oil']

In [558]:
def calculate_fitness_morning(morning_menu):
    point = 0
    point -= len(morning_menu)-len(set(item[9] for item in morning_menu))
    for item in morning_menu:
        point += 1 if (item[9] in G1) else 0
        point += 1 if (item[9] in G2) else 0
        point += 1 if (item[9] in G6) else 0
        point += 1 if (item[9] in G5) else 0
        point += 1 if (item[9] in G7) else 0
        point += 1 if (item[9] in G8) else 0
        point += 1 if (item[9] in G9) else 0

    return point

In [559]:
def calculate_fitness_lunch(lunch_menu):
    point = 0
    point -= len(lunch_menu)-len(set(item[9] for item in lunch_menu))
    for item in lunch_menu:     
        point += 1 if (item[9] in G2) else 0
        point += 1 if (item[9] in G3) else 0
        point += 1 if (item[9] in G5) else 0
        point += 1 if (item[9] in G6) else 0
        point += 1 if (item[9] in G8) else 0
        
    return point
    

In [560]:
def calculate_fitness_dinner(dinner_menu):
    point = 0
    point -= len(dinner_menu)-len(set(item[9] for item in dinner_menu))
    for item in dinner_menu:
        point += 1 if (item[9] in G2) else 0
        point += 1 if (item[9] in G3) else 0
        point += 1 if (item[9] in G5) else 0
        point += 1 if (item[9] in G6) else 0
        point += 1 if (item[9] in G8) else 0
        point += 1 if (item[9] in G9) else 0
            
    return point

In [561]:
def calculate_fitness_snack(snack_menu):
    point = 0
    point -= len(snack_menu)-len(set(item[9] for item in snack_menu))
    for item in snack_menu:
        point += 1 if (item[9] in G1) else 0
        point += 1 if (item[9] in G5) else 0
        point += 1 if (item[9] in G6) else 0
        point += 1 if (item[9] in G7) else 0
        point += 1 if (item[9] in G9) else 0
            
    return point

In [562]:
def calculate_fitness_calories(menu):
    point = 0 
    total_calo = sum(float(row[3]) for row in menu)
    
    if(total_calo >= tdee):
        if(aim == 'Tăng cân'):
            point += 2
        elif(aim == 'Giảm cân'):
            point -= 2
    else:
        if(aim == 'Tăng cân'):
            point -= 2
        elif(aim == 'Giảm cân'):
            point += 2
        
    return point
    

In [563]:
def calculate_fitness_favorite(menu):
    point = 0
    flattened_menu = [item[0] for item in menu]
    for item in flattened_menu:
        point += 1 if (item in favourite) else 0
            
    return point 

In [564]:
def calculate_fitness_nutrient(menu):
    point = 0
    #constraint nutrient
    protein_a_day = weight * 1.2 
    fat_a_day = tdee * (25/100) / 9
    sal_fat_a_day = tdee * (7/100) / 9
    fiber_a_day = weight * 38
    carbs_a_day = tdee * (35/100) / 4
    
    total_protein = sum(float(row[4]) for row in menu)
    total_fat = sum(float(row[5]) for row in menu)
    total_sat_fat = sum(float(row[6]) for row in menu)
    total_fiber = sum(float(row[7]) for row in menu)
    total_carbs = sum(float(row[8]) for row in menu)

    point += 1 if (abs(total_protein - protein_a_day) < 30) else -1
    point += 1 if (total_fat < fat_a_day) else -1
    point += 1 if (total_sal_fat < sal_fat_a_day) else -1
    point += 1 if (total_carbs < carbs_a_day) else -1
    
    return point 

In [565]:
def mutated_genes():
    length_data = len(data_list); 
    random_index = random_num(0, length_data-1); 
    return data_list[random_index]; 


In [566]:
def calculate_fitness(menu):
    point = 0 
    morning_group = []
    lunch_group = []
    dinner_group = []
    snack_group = []

    for index, item in enumerate(menu):
        if index < num_group_morning:
            morning_group.append(item)
        elif index < num_group_morning + num_group_lunch:
            lunch_group.append(item)
        elif index < num_group_morning + num_group_lunch + num_group_dinner:
            dinner_group.append(item)
        else:
            snack_group.append(item)
    
    point += calculate_fitness_morning(morning_group)
    point += calculate_fitness_lunch(lunch_group)
    point += calculate_fitness_dinner(dinner_group)
    point += calculate_fitness_snack(snack_group)
    point += calculate_fitness_calories(menu)
    point += calculate_fitness_favorite(menu)
    point += calculate_fitness_nutrient(menu)
    
    return point

In [567]:
def create_menu():
    menu_list = []
    for i in range(total_group):
        menu_list.append(mutated_genes())
    return menu_list


In [568]:
#test
total_protein2 = 0
menu = create_menu()


In [569]:
def create_init_population():
    return [create_menu() for _ in range(population_size)]

In [570]:
def select_parents(population, fitness_scores):

    selected_parents = []
    
    sorted_indices = np.argsort(fitness_scores)[::-1].tolist()
    selected_population = [population[i] for i in sorted_indices[:population_size // 2]]
    fitness_scores2 = [calculate_fitness(menu) for menu in selected_population]
    
    return selected_population

In [571]:
def crossover(parent1, parent2):
    crossover_point = random.randint(1, total_group - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    
    return child1, child2

In [572]:
def mutate(menu):
    for i in range(total_group):
        if random.uniform(0, 1) < mutation_rate:
            menu[i] = random.choice(data_list)
    return menu


In [573]:
def print_menu(menu):
    total_calo = 0
    morning_group = []
    lunch_group = []
    dinner_group = []
    snack_group = []

    for index, item in enumerate(menu):
        if index < num_group_morning:
            morning_group.append(item)
        elif index < num_group_morning + num_group_lunch:
            lunch_group.append(item)
        elif index < num_group_morning + num_group_lunch + num_group_dinner:
            dinner_group.append(item)
        else:
            snack_group.append(item)
            
    print('Bữa sáng: ', morning_group)
    print('Bữa trưa: ', lunch_group)
    print('Bữa tối: ', dinner_group)
    print('Bữa nhẹ: ', snack_group)
    print('point: ', calculate_fitness(menu))
    
    for row in menu:
        total_calo += int(row[3])
    print('Total calories: ', total_calo)
    
    
    

In [574]:
def genetic_algorithm():
    population = create_init_population()

    for generation in range(num_generations):
        fitness_scores = [calculate_fitness(menu) for menu in population]
        selected_parents = select_parents(population, fitness_scores)

        children = []
        for _ in range(population_size):
            parent1, parent2 = random.sample(selected_parents, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            children.extend([child1, child2])

        population = children
        
    best_solution = max(population, key=calculate_fitness)
    
    return best_solution

In [579]:
optimized_menu = genetic_algorithm()
# print("Optimized Menu:", optimized_menu)
print_menu(optimized_menu)

Bữa sáng:  [['Apple juice canned', '1 cup', 250.0, 125.0, 0.0, 0.0, 0.0, 0.0, 34.0, 'Fruits A-F'], ['Cream cheese', '1 oz.', 28.0, 105.0, 2.0, 11.0, 10.0, 0.0, 1.0, 'Dairy products'], ['Persimmons', '1 med.', 125.0, 75.0, 1.0, 0.0, 0.0, 2.0, 20.0, 'Fruits G-P']]
Bữa trưa:  [['Fresh, raw', '1 med.', 114.0, 35.0, 1.0, 0.0, 0.0, 0.6, 10.0, 'Fruits G-P'], ['Apple vinegar', '1/3 cup', 100.0, 14.0, 0.0, 0.0, 0.0, 0.0, 3.0, 'Fruits A-F'], ['Shredded wheat biscuit', '1', 28.0, 100.0, 3.0, 1.0, 0.0, 0.7, 23.0, 'Breads, cereals, fastfood,grains'], ['Salmon', '3 oz.', 85.0, 120.0, 17.0, 5.0, 1.0, 0.0, 0.0, 'Fish, Seafood']]
Bữa tối:  [['Peppers canned', '1 pod', 38.0, 10.0, 0.0, 0.0, 0.0, 0.0, 2.0, 'Vegetables R-Z'], ['Beef', '3 oz.', 85.0, 245.0, 23.0, 16.0, 15.0, 0.0, 0.0, 'Meat, Poultry'], ['Beans', '1 cup', 125.0, 25.0, 1.0, 0.0, 0.0, 0.8, 6.0, 'Vegetables A-E'], ['Apple juice canned', '1 cup', 250.0, 125.0, 0.0, 0.0, 0.0, 0.0, 34.0, 'Fruits A-F']]
Bữa nhẹ:  [['Beans', '1 cup', 125.0, 25.0, 1